In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
import os

In [ ]:
# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Augmentation pipeline for training
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),   # Resize to 224x224
    transforms.RandomHorizontalFlip(),                     # Horizontal flip
    transforms.ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.5), saturation=(0.8, 1.3), hue=(-0.05, 0.05)),  # Color jitter
    transforms.RandomRotation(15),                         # Random rotation
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value='random'),  # Random erasing (Cutout)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization
])

# Evaluation pipeline without augmentations
eval_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Load the training dataset
train_dataset = ImageFolder(root='./train', transform=train_transforms)
print("Class-to-Index Mapping:", train_dataset.class_to_idx)
print("Number of classes:", len(train_dataset.classes))

batch_size = 32
dropout_rate = 0.3

In [ ]:
# Create DataLoaders for training and evaluation
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
eval_dataset = ImageFolder(root='./test', transform=eval_transforms)
eval_loader = DataLoader(dataset=eval_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

In [3]:
# Model definition
class CombinedModel(nn.Module):
    def __init__(self, dropout):
        super(CombinedModel, self).__init__()

        # Load pre-trained models (ResNet50 and EfficientNet B2)
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        self.efficientnet = models.efficientnet_b2(weights=models.EfficientNet_B2_Weights.IMAGENET1K_V1)

        # Unfreeze all layers in both pre-trained models
        for param in self.resnet.parameters():
            param.requires_grad = True
        for param in self.efficientnet.parameters():
            param.requires_grad = True

        # Remove final classification layers from both models
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])
        self.efficientnet = nn.Sequential(*list(self.efficientnet.children())[:-1])

        # Batch normalization layer for the combined feature map
        self.bn = nn.BatchNorm1d(2048 + 1408)  # ResNet50 has 2048 features, EfficientNet B2 has 1408 features

        # Fully connected layers for classification (34 classes)
        self.fc = nn.Sequential(
            nn.Linear(2048 + 1408, 2048),  # Concatenate feature vectors
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(2048, 34)  # Output layer for 34 classes
        )

    def forward(self, x):
        # Extract features from both models
        resnet_features = self.resnet(x).view(x.size(0), -1)  # Flatten the ResNet features
        efficientnet_features = self.efficientnet(x).view(x.size(0), -1)  # Flatten the EfficientNet features

        # Concatenate features
        combined_features = torch.cat((resnet_features, efficientnet_features), dim=1)

        # Apply batch normalization and ReLU activation after concatenation
        combined_features = self.bn(combined_features)
        combined_features = torch.relu(combined_features)

        # Pass through fully connected layers
        output = self.fc(combined_features)
        return output



Class-to-Index Mapping: {'Baked Potato': 0, 'Crispy Chicken': 1, 'Donut': 2, 'Fries': 3, 'Hot Dog': 4, 'Sandwich': 5, 'Taco': 6, 'Taquito': 7, 'apple_pie': 8, 'burger': 9, 'butter_naan': 10, 'chai': 11, 'chapati': 12, 'cheesecake': 13, 'chicken_curry': 14, 'chole_bhature': 15, 'dal_makhani': 16, 'dhokla': 17, 'fried_rice': 18, 'ice_cream': 19, 'idli': 20, 'jalebi': 21, 'kaathi_rolls': 22, 'kadai_paneer': 23, 'kulfi': 24, 'masala_dosa': 25, 'momos': 26, 'omelette': 27, 'paani_puri': 28, 'pakode': 29, 'pav_bhaji': 30, 'pizza': 31, 'samosa': 32, 'sushi': 33}
Number of classes: 34


In [4]:
# Instantiate the model
model = CombinedModel(dropout=dropout_rate).to(device)

# Hyperparameters
num_epochs = 10
lr = 0.001

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam([
    {'params': model.resnet.parameters(), 'lr': lr * 0.1, 'weight_decay': 1e-3},
    {'params': model.efficientnet.parameters(), 'lr': lr * 0.1, 'weight_decay': 1e-3},
    {'params': model.fc.parameters(), 'lr': lr, 'weight_decay': 1e-3}
])

# Scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

In [5]:
# Training function
def train_model(model, train_loader, eval_loader, criterion, optimizer, num_epochs):
    best_f1 = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        total = 0
        all_preds = []
        all_labels = []

        with tqdm(total=len(train_loader), desc=f"Train Epoch [{epoch+1}/{num_epochs}]") as pbar:
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                total += labels.size(0)

                pbar.update(1)

        # Calculate metrics
        epoch_loss = running_loss / total
        epoch_f1 = f1_score(all_labels, all_preds, average='macro')
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train F1: {epoch_f1:.4f}")

        # Evaluate on validation set
        eval_loss, eval_acc, eval_f1 = evaluate_model(model, eval_loader, criterion, epoch, num_epochs)
        scheduler.step(eval_loss)

        if eval_f1 > best_f1:
            best_f1 = eval_f1
            print(f"Best model saved with Eval F1: {best_f1:.4f}")
            torch.save(model.state_dict(), 'best_model_34_classes.pth')

In [6]:
# Evaluation function
def evaluate_model(model, eval_loader, criterion, epoch, num_epochs):
    model.eval()
    all_preds = []
    all_labels = []
    correct = 0
    total = 0
    running_loss = 0.0

    with tqdm(total=len(eval_loader), desc=f"Eval Epoch [{epoch+1}/{num_epochs}]") as pbar:
        with torch.no_grad():
            for inputs, labels in eval_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                _, predicted = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)

                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
                pbar.update(1)

    eval_loss = running_loss / total
    eval_acc = correct / total
    eval_f1 = f1_score(all_labels, all_preds, average='macro')

    print(f"Eval Loss: {eval_loss:.4f}, Eval Accuracy: {eval_acc:.4f}, Eval F1: {eval_f1:.4f}")
    return eval_loss, eval_acc, eval_f1

In [7]:
# Run the training and evaluation
train_model(model, train_loader, eval_loader, criterion, optimizer, num_epochs)

Train Epoch [1/10]: 100%|██████████| 597/597 [01:52<00:00,  5.32it/s]


Epoch [1/10], Train Loss: 1.1947, Train F1: 0.6295


Eval Epoch [1/10]: 100%|██████████| 150/150 [00:29<00:00,  5.06it/s]


Eval Loss: 0.6433, Eval Accuracy: 0.8142, Eval F1: 0.8148
Best model saved with Eval F1: 0.8148


Train Epoch [2/10]: 100%|██████████| 597/597 [02:06<00:00,  4.74it/s]


...

Eval Loss: 0.3541, Eval Accuracy: 0.9102, Eval F1: 0.9196


Train Epoch [10/10]: 100%|██████████| 597/597 [03:46<00:00,  2.64it/s]


Epoch [10/10], Train Loss: 0.1997, Train F1: 0.9417


Eval Epoch [10/10]: 100%|██████████| 150/150 [00:36<00:00,  4.14it/s]


Eval Loss: 0.3009, Eval Accuracy: 0.9250, Eval F1: 0.9361
Best model saved with Eval F1: 0.9361
